# SBS V2/V3 AI-Powered Mapping System
## Interactive Notebook for Healthcare Provider Price List Integration

This notebook provides an interactive interface for:
- Loading SBS V2/V3 codes and your price list
- AI-powered matching based on service descriptions
- Generating mapping reports
- Reviewing and validating results

## 1. Setup and Imports

In [ ]:
import pandas as pd
import numpy as np
from sbs_ai_mapping_system import SBSMappingEngine, MappingValidator, MatchConfidence
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

print("✓ Imports successful")

## 2. Initialize the Mapping Engine

In [ ]:
# Initialize the engine
engine = SBSMappingEngine()

print("✓ Mapping engine initialized")

## 3. Load SBS V2 to V3 Mapping

Load the official CHI mapping file

In [ ]:
# Update this path to your actual file location
V2_V3_MAPPING_FILE = '/mnt/project/SBS_V2_to_V3_Map.xlsx'

v2_v3_map = engine.load_sbs_v2_v3_mapping(V2_V3_MAPPING_FILE)

if v2_v3_map is not None:
    print(f"\n✓ Loaded {len(v2_v3_map)} V2-V3 mappings")
    print(f"\nColumns available: {list(v2_v3_map.columns)}")
    print(f"\nFirst few rows:")
    display(v2_v3_map.head())
else:
    print("✗ Failed to load V2-V3 mapping")

## 4. Load Your Price List

**Important**: Update the file path and column names to match your price list format

In [ ]:
# Configure these variables for your price list
PRICELIST_FILE = 'YOUR_PRICELIST.xlsx'  # Update this path
CODE_COLUMN = 'Code'                     # Update to your code column name
DESCRIPTION_COLUMN = 'Description'       # Update to your description column name
PRICE_COLUMN = 'Price'                   # Update to your price column name

price_list = engine.load_price_list(
    PRICELIST_FILE,
    code_column=CODE_COLUMN,
    description_column=DESCRIPTION_COLUMN,
    price_column=PRICE_COLUMN
)

if price_list is not None:
    print(f"\n✓ Loaded {len(price_list)} items from price list")
    print(f"\nColumns: {list(price_list.columns)}")
    print(f"\nSample data:")
    display(price_list.head())
else:
    print("✗ Failed to load price list")
    print("Please update PRICELIST_FILE and column names above")

## 5. Configure Mapping Parameters

Specify which columns to use for mapping

In [ ]:
# Update these to match your V2-V3 mapping file columns
SBS_CODE_COLUMN = 'V3_Code'              # Column containing SBS V3 codes
SBS_DESCRIPTION_COLUMN = 'V3_Description' # Column containing SBS V3 descriptions
MIN_SIMILARITY_THRESHOLD = 0.60           # Minimum 60% similarity to consider a match

print(f"Mapping Configuration:")
print(f"  SBS Code Column: {SBS_CODE_COLUMN}")
print(f"  SBS Description Column: {SBS_DESCRIPTION_COLUMN}")
print(f"  Minimum Similarity: {MIN_SIMILARITY_THRESHOLD:.0%}")

## 6. Run the Mapping Process

This may take several minutes depending on the size of your datasets

In [ ]:
print("Starting mapping process...\n")

mapping_results = engine.map_to_price_list(
    sbs_df=v2_v3_map,
    sbs_code_col=SBS_CODE_COLUMN,
    sbs_desc_col=SBS_DESCRIPTION_COLUMN,
    min_threshold=MIN_SIMILARITY_THRESHOLD
)

print(f"\n✓ Mapping complete!")
print(f"\nResults preview:")
display(mapping_results.head(10))

## 7. Generate Mapping Report

In [ ]:
report = engine.generate_mapping_report(
    mapping_results,
    output_path='mapping_report.json'
)

print("\n" + "="*60)
print("MAPPING REPORT SUMMARY")
print("="*60)
print(f"\nTotal Records: {report['total_records']:,}")
print(f"Matched Records: {report['matched_records']:,}")
print(f"Unmatched Records: {report['unmatched_records']:,}")
print(f"\nMatch Rate: {report['match_rate']:.1%}")
print(f"Average Similarity Score: {report['average_similarity_score']:.2%}")
print(f"\nHigh Confidence Matches: {report['high_confidence_matches']:,}")
print(f"High Confidence Rate: {report['high_confidence_rate']:.1%}")
print(f"\nConfidence Distribution:")
for confidence, count in sorted(report['confidence_distribution'].items()):
    pct = (count / report['total_records']) * 100
    print(f"  {confidence}: {count:,} ({pct:.1f}%)")

## 8. Visualize Results

In [ ]:
# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Confidence Distribution
confidence_data = mapping_results['confidence'].value_counts()
axes[0, 0].bar(range(len(confidence_data)), confidence_data.values)
axes[0, 0].set_xticks(range(len(confidence_data)))
axes[0, 0].set_xticklabels(confidence_data.index, rotation=45, ha='right')
axes[0, 0].set_title('Mapping Confidence Distribution')
axes[0, 0].set_ylabel('Number of Records')

# 2. Similarity Score Distribution
matched_only = mapping_results[mapping_results['similarity_score'] > 0]
axes[0, 1].hist(matched_only['similarity_score'], bins=20, edgecolor='black')
axes[0, 1].set_title('Similarity Score Distribution (Matched Records)')
axes[0, 1].set_xlabel('Similarity Score')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].axvline(x=0.9, color='r', linestyle='--', label='90% threshold')
axes[0, 1].legend()

# 3. Match vs No Match Pie Chart
match_counts = [
    len(mapping_results[mapping_results['similarity_score'] > 0]),
    len(mapping_results[mapping_results['similarity_score'] == 0])
]
axes[1, 0].pie(match_counts, labels=['Matched', 'No Match'], autopct='%1.1f%%', startangle=90)
axes[1, 0].set_title('Overall Match Rate')

# 4. Alternative Matches Available
alt_match_dist = mapping_results['alternative_matches'].value_counts().sort_index()
axes[1, 1].bar(alt_match_dist.index, alt_match_dist.values)
axes[1, 1].set_title('Number of Alternative Matches')
axes[1, 1].set_xlabel('Alternative Matches')
axes[1, 1].set_ylabel('Number of Records')

plt.tight_layout()
plt.savefig('mapping_visualization.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✓ Visualization saved as 'mapping_visualization.png'")

## 9. Identify Records Requiring Review

In [ ]:
review_list = MappingValidator.generate_review_list(
    mapping_results,
    output_path='items_for_manual_review.xlsx'
)

print(f"\n✓ {len(review_list)} records flagged for manual review")
print(f"\nReview list saved to: items_for_manual_review.xlsx")
print(f"\nReview criteria:")
print(f"  - Low confidence matches (<70%)")
print(f"  - Ambiguous matches (top 2 scores within 5%)")

print(f"\nSample records requiring review:")
display(review_list[[
    'sbs_code', 'sbs_description', 
    'matched_pricelist_code', 'matched_pricelist_description',
    'similarity_score', 'confidence', 'is_ambiguous'
]].head(20))

## 10. Filter Results by Confidence Level

In [ ]:
# High confidence matches (ready to use)
high_confidence = mapping_results[
    mapping_results['confidence'].isin([
        MatchConfidence.EXACT.value,
        MatchConfidence.HIGH.value
    ])
]

print(f"High Confidence Matches: {len(high_confidence)}")
print(f"\nThese mappings can be used with confidence:")
display(high_confidence[[
    'sbs_code', 'sbs_description',
    'matched_pricelist_code', 'matched_pricelist_description',
    'price', 'similarity_score', 'confidence'
]].head(20))

# Export high confidence matches
high_confidence.to_excel('high_confidence_mappings.xlsx', index=False)
print(f"\n✓ High confidence mappings exported to: high_confidence_mappings.xlsx")

## 11. Export All Results

In [ ]:
# Export complete mapping results
output_file = 'complete_mapping_results.xlsx'
mapping_results.to_excel(output_file, index=False)

print(f"✓ Complete mapping results exported to: {output_file}")
print(f"\nFiles generated:")
print(f"  1. {output_file} - Complete mapping results")
print(f"  2. high_confidence_mappings.xlsx - Ready-to-use mappings")
print(f"  3. items_for_manual_review.xlsx - Records requiring review")
print(f"  4. mapping_report.json - Statistical report")
print(f"  5. mapping_visualization.png - Visual charts")

## 12. Search and Test Individual Matches

Test the matching algorithm on specific descriptions

In [ ]:
# Test with a specific SBS description
test_description = "Comprehensive oral examination"  # Change this to test

print(f"Testing matches for: '{test_description}'\n")

matches = engine.find_best_match(
    test_description,
    engine.price_list,
    'pricelist_description',
    'pricelist_code',
    top_n=10,
    min_threshold=0.3
)

if matches:
    print(f"Found {len(matches)} potential matches:\n")
    for i, match in enumerate(matches, 1):
        print(f"{i}. Score: {match['similarity_score']:.2%}")
        print(f"   Code: {match.get('code', 'N/A')}")
        print(f"   Description: {match['description']}")
        print(f"   Price: {match.get('price', 'N/A')}")
        print()
else:
    print("No matches found")

## 13. Advanced: Batch Update Thresholds

Re-run mapping with different similarity thresholds

In [ ]:
# Compare results with different thresholds
thresholds = [0.5, 0.6, 0.7, 0.8, 0.9]
threshold_comparison = []

for threshold in thresholds:
    results = engine.map_to_price_list(
        v2_v3_map,
        sbs_code_col=SBS_CODE_COLUMN,
        sbs_desc_col=SBS_DESCRIPTION_COLUMN,
        min_threshold=threshold
    )
    
    matched = len(results[results['similarity_score'] > 0])
    match_rate = matched / len(results)
    
    threshold_comparison.append({
        'threshold': threshold,
        'matches': matched,
        'match_rate': match_rate
    })

comparison_df = pd.DataFrame(threshold_comparison)
print("\nThreshold Comparison:")
display(comparison_df)

# Plot comparison
plt.figure(figsize=(10, 6))
plt.plot(comparison_df['threshold'], comparison_df['match_rate'] * 100, marker='o')
plt.xlabel('Similarity Threshold')
plt.ylabel('Match Rate (%)')
plt.title('Match Rate vs. Similarity Threshold')
plt.grid(True, alpha=0.3)
plt.savefig('threshold_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## Summary

You have successfully:
1. ✓ Loaded SBS V2/V3 mapping data
2. ✓ Loaded your healthcare provider price list
3. ✓ Mapped SBS codes to your price list using AI matching
4. ✓ Generated comprehensive reports and visualizations
5. ✓ Identified records requiring manual review
6. ✓ Exported results for use in your systems

### Next Steps:
1. Review the records in `items_for_manual_review.xlsx`
2. Validate high-confidence matches in `high_confidence_mappings.xlsx`
3. Import validated mappings into your billing system
4. Set up automated matching for new codes